## Step 1: Import the data to clean

In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od

In [3]:
dataset = 'https://www.kaggle.com/datasets/arianazmoudeh/airbnbopendata'

In [5]:
od.download(dataset)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: tigerchoco
Your Kaggle Key: ········


ApiException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Content-Length': '0', 'Date': 'Sun, 30 Jul 2023 18:45:13 GMT', 'Access-Control-Allow-Credentials': 'true', 'Set-Cookie': 'ka_sessionid=c1a405561528892e0fff6f00da242c76; max-age=2626560; path=/, GCLB=COPJgciQveajQg; path=/; HttpOnly', 'Turbolinks-Location': 'https://www.kaggle.com/api/v1/datasets/download/arianazmoudeh/airbnbopendata?datasetVersionNumber=None', 'Strict-Transport-Security': 'max-age=63072000; includeSubDomains; preload', 'Content-Security-Policy': "object-src 'none'; script-src 'nonce-D9HeS/0D2RvV6a1BKrN1qQ==' 'report-sample' 'unsafe-inline' 'unsafe-eval' 'strict-dynamic' https: http:; base-uri 'none'; report-uri https://csp.withgoogle.com/csp/kaggle/20201130; frame-src 'self' https://www.kaggleusercontent.com https://www.youtube.com/embed/ https://polygraph-cool.github.io https://www.google.com/recaptcha/ https://form.jotform.com https://submit.jotform.us https://submit.jotformpro.com https

In [ ]:
import os

In [ ]:
dataname = './airbnbopendata'

In [ ]:
os.listdir(dataname)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('airbnbopendata/Airbnb_Open_Data.csv', low_memory=False)
#or "engine='python"

In [ ]:
df

## Step 2: Cleanup

In [ ]:
df.info()

In [ ]:
df.isnull().mean()

In [ ]:
#drop columns with more than 50% missing values
print(df.columns[df.isnull().mean() > 0.5], '\n')

In [ ]:
df = df.drop(df.columns[df.isnull().mean() > 0.5], axis=1)

In [ ]:
#Remove unnecessary columns
df.drop(columns=['host id','country','country code'],axis=1,inplace=True)

In [ ]:
#correcting columns' names
df.columns=[col.lower().replace(' ','_') for col in df.columns]
df.columns

In [ ]:
df

In [ ]:
#counting borough and correcting data names
df['neighbourhood_group'].value_counts()

In [ ]:
df['neighbourhood_group'] = df['neighbourhood_group'].replace('brookln', 'Brooklyn')
df['neighbourhood_group'] = df['neighbourhood_group'].replace('manhatan', 'Manhattan')

In [ ]:
df['neighbourhood_group'].value_counts()

In [ ]:
df['price']

In [ ]:
#removing non decimal digit
df['price'] = df['price'].str.replace(r'\D','',regex=True)
df['service_fee'] = df['service_fee'].str.replace(r'\D','',regex=True)

In [ ]:
df.isnull().sum()

In [ ]:
#filling the nulls in column (dealing missing data)
for column in df.columns:
    if df[column].isnull().sum() != 0:
        df[column] = df[column].fillna(df[column].mode()[0])

In [ ]:
df.isnull().sum()

In [ ]:
#changing digit data from object to integer/float
convert = {'construction_year': int, 'price': int, 'service_fee': int, 'review_rate_number': float}

df = df.astype(convert)
df.dtypes

In [ ]:
df.to_csv('airbnb_python.csv')

In [ ]:
df['room_type'].value_counts()

## Step 3: Statistical Analysis

In [ ]:
#finding out the relevance between neighbourhoods and prices
df_n_roomtype = df.groupby(['neighbourhood','room_type']).agg({'price':'sum'})
df_n_roomtype

In [ ]:
df_n_roomtype = pd.pivot_table(df_n_roomtype, values='price', index=['neighbourhood'], columns=['room_type'])

In [ ]:
#top 3 highest revenue in Entire home/apt
df_n_roomtype.sort_values('Entire home/apt', ascending = False).head(3)

In [ ]:
#top 3 lowest revenue in Shared room
df_n_roomtype.sort_values('Shared room', ascending = True).head(3)

## Step 4: Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
#rental units of different boroughs
rm = df['neighbourhood_group'].value_counts() 
fig = px.bar(y = rm.values,
             x = rm.index,
             color = rm.index,
             color_discrete_sequence=px.colors.qualitative.G10,
             text = rm.values,
             title = 'Rentals in Different Neighbourhood Group',
             template = 'plotly_dark'
             )
fig.update_layout(
    xaxis_title = 'Borough',
    yaxis_title = 'Rental Unit',
    font = dict(size=16,family='Courier New')
    )
fig.show()

In [ ]:
#numbers of room types
rm = df['room_type'].value_counts() 
fig = px.bar(y = rm.values,
             x = rm.index,
             color = rm.index,
             color_discrete_sequence=px.colors.qualitative.Pastel1,
             text = rm.values,
             title = 'Numbers of Room Types',
             template = 'plotly_dark'
             )
fig.update_layout(
    xaxis_title = 'Room Type',
    yaxis_title = 'Count',
    font = dict(size=16,family='Courier New')
    )
fig.show()

In [ ]:
#finding out Airbnb top 10 neighbourhoods
plt.figure(figsize=(10,5))
sns.barplot(x=df['neighbourhood'].value_counts()[0:10], 
            y=df['neighbourhood'].value_counts().index[0:10], palette='autumn')
plt.xlabel('Rental Unit')
plt.ylabel('Neighbourhood')
plt.show()

## Step 5: Export the clean data to a databa

In [ ]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('airbnb_python.db')
df2 = pd.read_csv("airbnb_python.csv")
df2.to_sql('airbnb_python',conn,if_exists='append',index=False)
pd.read_sql('select * from airbnb_python', conn)
